In [1]:
# 통계적으로 가중치를 구한 후 두 단어 간의 유사도를 단어 간의 연관도로 적용하는 방법
# 1. 단어마다 가중치를 할당해야 함(출현 빈도, TF-IDF 등으로 계산)
# 2. 단어 간의 유사도 계산(cosine similarity 등의 방법)

In [2]:
import glob

pos_review=(glob.glob('d:/data/imdb/train/pos/*.txt'))[0:100] # 긍정 리뷰 100개 로딩

lines_pos=[]
for i in pos_review:
    try:
        f=open(i, 'r')
        temp=f.readlines()[0]
        lines_pos.append(temp)
        f.close()
    except Exception as e:
        continue

len(lines_pos)

100

In [3]:
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

tokenizer=RegexpTokenizer('[\w]+')
stop_words=stopwords.words('english')
# TF-IDF 가중치 할당
vec=TfidfVectorizer(stop_words=stop_words)
vector_lines_pos=vec.fit_transform(lines_pos)
A=vector_lines_pos.toarray()
print(A.shape)
print(A)
# x축 단어, y축 문서

(100, 4001)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.06538462 0.         ... 0.         0.         0.        ]
 [0.         0.23078109 0.         ... 0.         0.         0.        ]]


In [4]:
# 현재 상태는 100개의 문서의 유사도
# 단어 간의 유사도를 구하는 것이 목적이므로 단어-문서 행렬로 변경
A=A.transpose()

print(A.shape)
print(A)
# x축 문서, y축 단어

(4001, 100)
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.06538462 0.23078109]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [5]:
import numpy as np
from scipy import sparse
# 밀집행렬(dense array)
a=np.array([[0.5,0,0],[0,1,0],[0.7,0,1.5]])
# 밀집행렬을 희소행렬(sparse array)로 변환
# 밀집행렬의 단점 : 0이 많을 경우 메모리 낭비가 심함
# 희소행렬은 0이 아닌 값들의 위치와 값만 기록하여 메모리를 절약하는 방식
b=sparse.csr_matrix(a)
print(b)
# (0, 0) 0.5 → 인덱스 0,0에 값 0.5
# (1, 1) 1.0 → 인덱스 1,1에 값 1.0
# (2, 0) 0.7 → 인덱스 2,0에 값 0.7
# (2, 2) 1.5 → 인덱스 2,2에 값 1.5
c=b.toarray()
print(c)

  (0, 0)	0.5
  (1, 1)	1.0
  (2, 0)	0.7
  (2, 2)	1.5
[[0.5 0.  0. ]
 [0.  1.  0. ]
 [0.7 0.  1.5]]


In [6]:
from sklearn.metrics.pairwise import cosine_similarity

A_sparse=sparse.csr_matrix(A)
similarities_sparse=cosine_similarity(A_sparse, dense_output=False) # 코사인 유사도 계산
list(similarities_sparse.todok().items())[35000:35010] # todok() : 행렬을 딕셔너리 형태로 변환

# 단어 이름이 아닌 인덱스 형태로 출력
# ((1469, 108), 0.37803585968894865) → 1469 단어와 108 단어의 유사도는 37%

[((1469, 108), 0.37803585968894865),
 ((1470, 108), 0.2189685434746738),
 ((1476, 108), 0.06407477897013734),
 ((1477, 108), 0.185189577514238),
 ((1480, 108), 0.20111036876169444),
 ((1489, 108), 0.06995711757772019),
 ((1496, 108), 0.10714874067068783),
 ((1503, 108), 0.30487333830091773),
 ((1504, 108), 0.30487333830091773),
 ((1512, 108), 0.30487333830091773)]

In [7]:
print(vec.get_feature_names()[1469])
print(vec.get_feature_names()[108])
# fraud와 actual의 유사도는 37.8%

fraud
actual


In [8]:
import pandas as pd
import numpy as np

# 위의 결과값을 데이터프레임으로 출력
df=pd.DataFrame(list(similarities_sparse.todok().items()), columns=['words','weight'])
df2=df.sort_values(by=['weight'], ascending=False)
df2=df2.reset_index(drop=True)
df3=df2.loc[np.round(df2['weight'])<1] # 자기 자신과의 짝은 1이 되므로 1 미만의 항목들만 출력
df3=df3.reset_index(drop=True)
df3.head(10)

,words,weight
0,"(3971, 3372)",0.499961
1,"(3372, 3971)",0.499961
2,"(1192, 2554)",0.499958
3,"(2554, 1192)",0.499958
4,"(2468, 1321)",0.499957
5,"(2468, 710)",0.499957
6,"(710, 2468)",0.499957
7,"(1321, 2468)",0.499957
8,"(2146, 889)",0.499909
9,"(889, 2146)",0.499909


In [9]:
print(vec.get_feature_names()[3971])
print(vec.get_feature_names()[3372])

writers
stop
